## import libraries

In [392]:
import cv2
import tensorly as tl
import numpy as np

## reading the video

In [393]:
source = cv2.VideoCapture('ghasedak.mp4')

## number of frames

In [394]:
video_frame = int(source.get(cv2.CAP_PROP_FRAME_COUNT))
video_frame

129

## convert video to tensor

In [395]:
frames_array=[]
frame_nb=0
while source.isOpened() and frame_nb < video_frame:
    ret, frame = source.read()
    if not ret:
        break
    frames_array.append(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frame_nb += 1
source.release()
cv2.destroyAllWindows()

In [396]:
vid_tensor = tl.tensor(frames_array)
vid_tensor

array([[[[221, 221, 221],
         [221, 221, 221],
         [221, 221, 221],
         ...,
         [ 90,  90,  90],
         [ 90,  90,  90],
         [ 90,  90,  90]],

        [[221, 221, 221],
         [221, 221, 221],
         [221, 221, 221],
         ...,
         [ 90,  90,  90],
         [ 90,  90,  90],
         [ 90,  90,  90]],

        [[221, 221, 221],
         [221, 221, 221],
         [221, 221, 221],
         ...,
         [ 90,  90,  90],
         [ 90,  90,  90],
         [ 90,  90,  90]],

        ...,

        [[ 78,  78,  78],
         [ 78,  78,  78],
         [ 79,  79,  79],
         ...,
         [ 61,  61,  61],
         [ 59,  59,  59],
         [ 58,  58,  58]],

        [[ 78,  78,  78],
         [ 78,  78,  78],
         [ 79,  79,  79],
         ...,
         [ 60,  60,  60],
         [ 58,  58,  58],
         [ 57,  57,  57]],

        [[ 76,  76,  76],
         [ 76,  76,  76],
         [ 78,  78,  78],
         ...,
         [ 60,  60,  60],
        

In [397]:
vid_ten=vid_tensor[:,:,:,0]

In [398]:
v_shape=vid_ten.shape
v_shape

(129, 336, 592)

In [399]:
frame_size = (592,336)

## HOSVD Decomposition

### unfold 1 of tensor of video

In [405]:
unfold1=tl.unfold(vid_ten, 0)
unfold1.shape

(129, 198912)

### unfold 2 of tensor of video

In [406]:
unfold2=tl.unfold(vid_ten, 1)
unfold2.shape

(336, 76368)

### unfold 3 of tensor of video

In [407]:
unfold3=tl.unfold(vid_ten, 2)
unfold3.shape

(592, 43344)

### SVD docomposition on unfold 1 , unfold 2 and unfold 3 for computing U1,U2 and U3

In [408]:
U1, s1, V1 = np.linalg.svd(unfold1,full_matrices=False)
U2, s2, V2 = np.linalg.svd(unfold2,full_matrices=False)
U3, s3, V3 = np.linalg.svd(unfold3,full_matrices=False)

### computing S

In [409]:
A1=tl.tenalg.mode_dot(vid_ten, U1.T, 0)
A2=tl.tenalg.mode_dot(A1, U2.T, 1)
S=tl.tenalg.mode_dot(A2, U3.T, 2)

In [410]:
U1.shape

(129, 129)

In [411]:
U2.shape

(336, 336)

In [412]:
U3.shape

(592, 592)

In [413]:
S.shape

(129, 336, 592)

## reconstruct the tensor with lower dimentions

### 20,50,50

In [414]:
T_reconst=np.zeros(v_shape)
T_d1=tl.tenalg.mode_dot(S[:20,:50,:50], U1[:,:20], 0)
T_d2=tl.tenalg.mode_dot(T_d1, U2[:,:50], 1)
T_reconst=tl.tenalg.mode_dot(T_d2, U3[:,:50], 2)

In [415]:
T_reconst.shape

(129, 336, 592)

In [416]:
T_reconst = T_reconst.astype(np.uint8)

### convert tensor to video

In [417]:
output1 = cv2.VideoWriter('compress1.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, frame_size,0)

In [418]:
for i in range(0,129):
    output1.write(T_reconst[i])
output1.release()

### 50,70,100

In [419]:
T_reconst=np.zeros(v_shape)
T_d1=tl.tenalg.mode_dot(S[:50,:70,:100], U1[:,:50], 0)
T_d2=tl.tenalg.mode_dot(T_d1, U2[:,:70], 1)
T_reconst=tl.tenalg.mode_dot(T_d2, U3[:,:100], 2)
T_reconst = T_reconst.astype(np.uint8)

### convert tensor to video

In [420]:
output2 = cv2.VideoWriter('compress2.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, frame_size,0)

In [421]:
for i in range(0,129):
    output2.write(T_reconst[i])
output2.release()

### 75,100,200

In [422]:
T_reconst=np.zeros(v_shape)
T_d1=tl.tenalg.mode_dot(S[:75,:100,:200], U1[:,:75], 0)
T_d2=tl.tenalg.mode_dot(T_d1, U2[:,:100], 1)
T_reconst=tl.tenalg.mode_dot(T_d2, U3[:,:200], 2)
T_reconst = T_reconst.astype(np.uint8)

### convert tensor to video

In [423]:
output3 = cv2.VideoWriter('compress3.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, frame_size,0)

In [424]:
for i in range(0,129):
    output3.write(T_reconst[i])
output3.release()

### 100,200,300

In [425]:
T_reconst=np.zeros(v_shape)
T_d1=tl.tenalg.mode_dot(S[:100,:200,:300], U1[:,:100], 0)
T_d2=tl.tenalg.mode_dot(T_d1, U2[:,:200], 1)
T_reconst=tl.tenalg.mode_dot(T_d2, U3[:,:300], 2)
T_reconst = T_reconst.astype(np.uint8)

### convert tensor to video

In [426]:
output4 = cv2.VideoWriter('compress4.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, frame_size,0)

In [427]:
for i in range(0,129):
    output4.write(T_reconst[i])
output4.release()